In [54]:
import QDNS
from QDNS import gates
import logging

In [55]:
class Alice(QDNS.Node):
    def __init__(self, *user_args):
        super().__init__("Alice")
        self.create_new_application(self.alice_default_app, *user_args)

    @staticmethod
    def alice_default_app(app: QDNS.Application, *user_args):
        key_length = user_args[0]
        message = user_args[1]
        
        protocol = app.run_qkd_protocol("Bob", key_length, QDNS.BB84_METHOD)
        if protocol is None:
            print("Protocol is failed to establish.")
        
        key, length = protocol
        encrypted = QDNS.string_encode(key, message)
        app.send_classic_data("Bob", encrypted)

In [56]:
class Bob(QDNS.Node):
    def __init__(self):
        super().__init__("Bob")
        self.create_new_application(self.bob_default_app)

    @staticmethod
    def bob_default_app(app: QDNS.Application):
        protocol = app.wait_qkd()
        if protocol is None:
            print("Protocol is failed to establish.")
            
        key, length = protocol
        encrypted = app.wait_next_package().data
        
        print("Bob recieves encrypted: ", encrypted)
        print("Bob decryptes: ", QDNS.string_decode(key, encrypted))

In [63]:
def main():
    logging.basicConfig(level=logging.WARNING)
    
    key_length = 512
    message = "Hello Bob! This is very very private message."
    alice, bob = Alice(key_length, message), Bob()
    
    net = QDNS.Network(alice, bob)
    net.add_channels(alice, bob, length=5.0)
    
    core_count = int(QDNS.core_count / 2)
    frames = {
        2: {
            1: int(key_length / core_count) + 16,
        }
    }
    backend_conf = QDNS.BackendConfiguration(QDNS.CIRQ_BACKEND, core_count, frames)
    
    sim = QDNS.Simulator()
    results = sim.simulate(net, backend_conf)

In [65]:
main()

Bob recieves encrypted:  wqPClcKMwpzCj1Biwp_CglFAwoXCiMKawpNRwonCpEDCp8KFwqPCmVHClsKVwpLCqkDCocKSwpnClsKSwpTClkDCncKFwqPCk8KRwofClk4=
Bob decryptes:  Hello Bob! This!is very verz prhvate message.
